# End-to-End Twitter Turkish Sentiment Analysis Application
## 2-Sentiment Analysis


### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/morethan101/')

### Install Packages

In [ ]:
!pip install transformers
!pip install emoji

### Import Packages

In [31]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import re
import emoji
from tqdm.notebook import tqdm
tqdm.pandas()

In [5]:
#df = pd.read_csv("data/deprem_hashtag_subat_2023_unique.csv")
df = pd.read_csv("data/chatgpt_2022-11-30_2022-12-30_unique.csv")

df

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL,OnlyDate
0,0,1608613358500220928,2022-12-29 23:58:05+00:00,ademmcansiz,ChatGPT ile vs atıyoruz olmuyor diyom hala anl...,0,0,3,0,https://twitter.com/ademmcansiz/status/1608613...,2022-12-29
1,1,1608585229756936192,2022-12-29 22:06:19+00:00,berkelevent,ChatGPT çıktı çıkalı Co-pilot’u da unuttuk..,0,0,0,0,https://twitter.com/berkelevent/status/1608585...,2022-12-29
2,2,1608581500060205057,2022-12-29 21:51:30+00:00,0xugurcaneser,Her türlü yazı işlerimi chatgpt yapıyor sağols...,0,0,0,0,https://twitter.com/0xugurcaneser/status/16085...,2022-12-29
3,3,1608580588247871488,2022-12-29 21:47:52+00:00,atiorh,@imTolstoyevski 2019 Haziran bölümlerinde yapa...,0,1,0,0,https://twitter.com/atiorh/status/160858058824...,2022-12-29
4,4,1608579005657272320,2022-12-29 21:41:35+00:00,nounpackage,Ben en güzel Siber Güvenlik muhabbetini ChatGP...,0,0,0,0,https://twitter.com/nounpackage/status/1608579...,2022-12-29
...,...,...,...,...,...,...,...,...,...,...,...
5578,5584,1598294973857792000,2022-12-01 12:36:31+00:00,DigitalReportTR,OpenAI’ın yeni chat robotu ChatGPT tanıtıldı\n...,0,1,1,0,https://twitter.com/DigitalReportTR/status/159...,2022-12-01
5579,5585,1598294149585047558,2022-12-01 12:33:14+00:00,venividi_trade,Yapay zeka üzerine araştırmalar yapan @OpenAI ...,5,3,14,1,https://twitter.com/venividi_trade/status/1598...,2022-12-01
5580,5586,1598293316843737088,2022-12-01 12:29:56+00:00,yigitkonur,Bugün mutlu musunuz? Biraz daha depresif hisse...,10,24,319,7,https://twitter.com/yigitkonur/status/15982933...,2022-12-01
5581,5587,1598288040052502528,2022-12-01 12:08:58+00:00,kisalar,Öykülerin tam da oturmuyor olmasının sebebi be...,1,0,0,0,https://twitter.com/kisalar/status/15982880400...,2022-12-01


### Sentiment Analysis Model init

In [6]:
#load model and tokenizer
#https://huggingface.co/savasy/bert-base-turkish-sentiment-cased
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
# create pipeline
sa = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

### Preprocessing

In [7]:
def preprocess_word(word):
    # Remove punctuation
    #word = word.strip('\'"?!,.():;')
    
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)|😉', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)

    return tweet

def remove_emoji(tweet):
    return emoji.replace_emoji(tweet, replace=" ")


def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    
    #Clean only digits
    tweet = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", tweet)
    
    # Replaces URLs with the word URL
    #tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', '', tweet)
    
    # Replace @handle with the word USER_MENTION
    #tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    tweet = re.sub(r'@[\S]+', '', tweet)
    
    # Replaces #hashtag with hashtag
    #tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    tweet = re.sub(r'#(\S+)', '', tweet)
    
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')

    # Replace emojis with either EMO_POS or EMO_NEG
    #tweet = handle_emojis(tweet)

    #remove emojis
    tweet = remove_emoji(tweet)
    
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
      word = preprocess_word(word)
      #if is_valid_word(word):
      #    processed_tweet.append(word)
      processed_tweet.append(word)

    return ' '.join(processed_tweet)

In [24]:
index = 3200
print(df.loc[index].Tweet)
p = sa(df.loc[index].Tweet)
print(p)

bu chatGPT muazzam bir iş olmuş yav ağzım açık kalıyo her seferinde
[{'label': 'positive', 'score': 0.9942097663879395}]


In [25]:
pre_twit = preprocess_tweet(df.loc[index].Tweet)
print(pre_twit)
p = sa(pre_twit)
print(p)

bu chatgpt muazzam bir iş olmuş yav ağzım açık kalıyo her seferinde
[{'label': 'positive', 'score': 0.9942097663879395}]


In [26]:
def get_sentiment_analysis(tweet):
    """True if tweet has positive compound sentiment, False otherwise."""
    p = sa(str(tweet))
    #print("label", p[0]["label"])
    #print("score", round(p[0]["score"],4))
    return pd.Series([p[0]["label"], round(p[0]["score"],4)])

In [27]:
s = get_sentiment_analysis(pre_twit)
print(s)

0    positive
1      0.9942
dtype: object


### Sentiment Analysis

In [32]:
df["sentiment_label"] = "-"
df["sentiment_score"] = -1
df[["sentiment_label", "sentiment_score"]] = df["Tweet"].progress_apply(get_sentiment_analysis)

  0%|          | 0/5583 [00:00<?, ?it/s]

In [35]:
df['sentiment_score'].apply(lambda x: float(x))

THRESHOLD = 0.95

In [36]:
df

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL,OnlyDate,sentiment_label,sentiment_score
0,0,1608613358500220928,2022-12-29 23:58:05+00:00,ademmcansiz,ChatGPT ile vs atıyoruz olmuyor diyom hala anl...,0,0,3,0,https://twitter.com/ademmcansiz/status/1608613...,2022-12-29,positive,0.8235
1,1,1608585229756936192,2022-12-29 22:06:19+00:00,berkelevent,ChatGPT çıktı çıkalı Co-pilot’u da unuttuk..,0,0,0,0,https://twitter.com/berkelevent/status/1608585...,2022-12-29,positive,0.5480
2,2,1608581500060205057,2022-12-29 21:51:30+00:00,0xugurcaneser,Her türlü yazı işlerimi chatgpt yapıyor sağols...,0,0,0,0,https://twitter.com/0xugurcaneser/status/16085...,2022-12-29,negative,0.8703
3,3,1608580588247871488,2022-12-29 21:47:52+00:00,atiorh,@imTolstoyevski 2019 Haziran bölümlerinde yapa...,0,1,0,0,https://twitter.com/atiorh/status/160858058824...,2022-12-29,negative,0.7501
4,4,1608579005657272320,2022-12-29 21:41:35+00:00,nounpackage,Ben en güzel Siber Güvenlik muhabbetini ChatGP...,0,0,0,0,https://twitter.com/nounpackage/status/1608579...,2022-12-29,positive,0.9589
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,5584,1598294973857792000,2022-12-01 12:36:31+00:00,DigitalReportTR,OpenAI’ın yeni chat robotu ChatGPT tanıtıldı\n...,0,1,1,0,https://twitter.com/DigitalReportTR/status/159...,2022-12-01,positive,0.7625
5579,5585,1598294149585047558,2022-12-01 12:33:14+00:00,venividi_trade,Yapay zeka üzerine araştırmalar yapan @OpenAI ...,5,3,14,1,https://twitter.com/venividi_trade/status/1598...,2022-12-01,positive,0.7511
5580,5586,1598293316843737088,2022-12-01 12:29:56+00:00,yigitkonur,Bugün mutlu musunuz? Biraz daha depresif hisse...,10,24,319,7,https://twitter.com/yigitkonur/status/15982933...,2022-12-01,positive,0.9908
5581,5587,1598288040052502528,2022-12-01 12:08:58+00:00,kisalar,Öykülerin tam da oturmuyor olmasının sebebi be...,1,0,0,0,https://twitter.com/kisalar/status/15982880400...,2022-12-01,positive,0.6335


In [41]:
df_positive = df[df["sentiment_label"]=="positive"]
#df_positive = df[(df["sentiment_label"]=="positive") & (df["sentiment_score"]>THRESHOLD)]
df_positive

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL,OnlyDate,sentiment_label,sentiment_score
0,0,1608613358500220928,2022-12-29 23:58:05+00:00,ademmcansiz,ChatGPT ile vs atıyoruz olmuyor diyom hala anl...,0,0,3,0,https://twitter.com/ademmcansiz/status/1608613...,2022-12-29,positive,0.8235
1,1,1608585229756936192,2022-12-29 22:06:19+00:00,berkelevent,ChatGPT çıktı çıkalı Co-pilot’u da unuttuk..,0,0,0,0,https://twitter.com/berkelevent/status/1608585...,2022-12-29,positive,0.5480
4,4,1608579005657272320,2022-12-29 21:41:35+00:00,nounpackage,Ben en güzel Siber Güvenlik muhabbetini ChatGP...,0,0,0,0,https://twitter.com/nounpackage/status/1608579...,2022-12-29,positive,0.9589
5,5,1608577851284156416,2022-12-29 21:37:00+00:00,BetulGymn,@dunyadanfinans https://t.co/v08AOyT0T7 Yerli ...,2,3,4,1,https://twitter.com/BetulGymn/status/160857785...,2022-12-29,positive,0.9945
6,6,1608577733147381760,2022-12-29 21:36:32+00:00,BetulGymn,@dunyadanfinans @mizahdesen https://t.co/v08AO...,0,0,10,0,https://twitter.com/BetulGymn/status/160857773...,2022-12-29,positive,0.9889
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,5584,1598294973857792000,2022-12-01 12:36:31+00:00,DigitalReportTR,OpenAI’ın yeni chat robotu ChatGPT tanıtıldı\n...,0,1,1,0,https://twitter.com/DigitalReportTR/status/159...,2022-12-01,positive,0.7625
5579,5585,1598294149585047558,2022-12-01 12:33:14+00:00,venividi_trade,Yapay zeka üzerine araştırmalar yapan @OpenAI ...,5,3,14,1,https://twitter.com/venividi_trade/status/1598...,2022-12-01,positive,0.7511
5580,5586,1598293316843737088,2022-12-01 12:29:56+00:00,yigitkonur,Bugün mutlu musunuz? Biraz daha depresif hisse...,10,24,319,7,https://twitter.com/yigitkonur/status/15982933...,2022-12-01,positive,0.9908
5581,5587,1598288040052502528,2022-12-01 12:08:58+00:00,kisalar,Öykülerin tam da oturmuyor olmasının sebebi be...,1,0,0,0,https://twitter.com/kisalar/status/15982880400...,2022-12-01,positive,0.6335


In [42]:
#df_positive.to_csv("data/deprem_hashtag_subat_2023_positive.csv")
df_positive.to_csv("data/chatgpt_2022-11-30_2022-12-30_positive.csv")

In [43]:
df_negative = df[df["sentiment_label"]=="negative"]
#df_negative = df[(df["sentiment_label"]=="negative") & (df["sentiment_score"]>THRESHOLD)]
df_negative

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL,OnlyDate,sentiment_label,sentiment_score
2,2,1608581500060205057,2022-12-29 21:51:30+00:00,0xugurcaneser,Her türlü yazı işlerimi chatgpt yapıyor sağols...,0,0,0,0,https://twitter.com/0xugurcaneser/status/16085...,2022-12-29,negative,0.8703
3,3,1608580588247871488,2022-12-29 21:47:52+00:00,atiorh,@imTolstoyevski 2019 Haziran bölümlerinde yapa...,0,1,0,0,https://twitter.com/atiorh/status/160858058824...,2022-12-29,negative,0.7501
7,7,1608576320002478081,2022-12-29 21:30:55+00:00,hey_uren,@dc_boyblue @hellstabber -Konuşmaya dahil oldu...,0,0,0,0,https://twitter.com/hey_uren/status/1608576320...,2022-12-29,negative,0.9990
8,8,1608576028284448781,2022-12-29 21:29:45+00:00,elfsensoz,ChatGPT ile asla ama asla adam asmaca oynamayın.,1,0,2,0,https://twitter.com/elfsensoz/status/160857602...,2022-12-29,negative,0.9975
11,11,1608567682491895808,2022-12-29 20:56:35+00:00,instayikapatti1,ChatGPT’nin saniyeler icerisinde istedigim kod...,0,0,0,0,https://twitter.com/instayikapatti1/status/160...,2022-12-29,negative,0.8747
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5554,5560,1598582991571255296,2022-12-02 07:41:00+00:00,Mehmetoloji_,Meslek değiştirip yazılıma başlayan jr. Develo...,0,0,6,0,https://twitter.com/Mehmetoloji_/status/159858...,2022-12-02,negative,0.9892
5570,5576,1598437869286309889,2022-12-01 22:04:20+00:00,MumiDemir,Bu chatgpt’nin gösterdiği gelişim inanılmaz fa...,0,0,0,0,https://twitter.com/MumiDemir/status/159843786...,2022-12-01,negative,0.9303
5571,5577,1598434841120616451,2022-12-01 21:52:18+00:00,KadirTunaYeni,@AkyuzUluc Yeterince ses getirmedi ama daha et...,0,0,1,1,https://twitter.com/KadirTunaYeni/status/15984...,2022-12-01,negative,0.9855
5574,5580,1598356172418990080,2022-12-01 16:39:42+00:00,hz_robot,OpenAI'nin en son yapay zeka(LM) uygulaması ol...,1,0,0,0,https://twitter.com/hz_robot/status/1598356172...,2022-12-01,negative,0.9983


In [44]:
#df_negative.to_csv("data/deprem_hashtag_subat_2023_negative.csv")
df_negative.to_csv("data/chatgpt_2022-11-30_2022-12-30_negative.csv")